In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [37]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [38]:
t_data=pd.read_csv('/content/drive/My Drive/AI/amex_2019/training.csv') #training data
l_data=pd.read_csv('/content/drive/My Drive/AI/amex_2019/leaderboard_dataset.csv')  #leaderboard data
pt_data=pd.read_csv('/content/drive/My Drive/AI/amex_2019/preprocessed_data.csv') #preprocessed training data
pl_data=pd.read_csv('/content/drive/My Drive/AI/amex_2019/preprocessed_label.csv') #preprocessed leaderboard data

In [39]:
t_data=t_data.fillna(-1)
# l_data=l_data.fillna(-1)

In [40]:
t_data['VAR14'].replace('.',np.nan, inplace = True)


# t_data['VAR14']=t_data['VAR14'].replace(np.nan,'1')
# t_data['VAR14']=pd.to_numeric(t_data['VAR14'])

# label encoding the data
le=LabelEncoder()
t_data['VAR21'] = le.fit_transform(t_data['VAR21'])
pt_data['VAR21'] = le.fit_transform(pt_data['VAR21'])
l_data['VAR14']=pd.to_numeric(t_data['VAR14'])

In [41]:
t_data['VAR14']

0          1
1          1
2          1
3          1
4        NaN
        ... 
33995      1
33996      1
33997      1
33998      1
33999      1
Name: VAR14, Length: 34000, dtype: object

In [42]:
X_train_t= t_data
X_train_p=pt_data


x_test=l_data

In [43]:

# from imblearn.over_sampling import SMOTE
# ​
# # Resample the minority class. You can change the strategy to 'auto' if you are not sure.
# sm = SMOTE(sampling_strategy='auto', random_state=7)
# ​
# # Fit the model to generate the data.
# oversampled_trainX, oversampled_trainY = sm.fit_sample(x_train, y_train)

In [44]:
X_train_p.head()

,VAR1,VAR2,VAR3,VAR4,VAR5,VAR6,VAR7,VAR8,VAR9,VAR10,VAR11,VAR12,VAR13,VAR14,VAR15,VAR16,VAR17,VAR18,VAR19,VAR20,VAR21
0,1,828.235294,0.138889,321.428571,625.911006,1.818182,1.081550,198.113469,53.988554,58.632548,0.279273,100.083403,1.000000,1,1.000000,1.540594,821.281092,1,0,100.104991,1
1,2,911.764706,0.027778,489.114574,611.574748,8.181818,1.344479,198.600020,22.086661,54.882308,4.446269,15.012510,9.088815,1,7.841057,1.614613,821.281092,0,1,146.654045,0
2,3,962.352941,0.833333,35.714286,615.825381,8.181818,0.720796,197.267767,53.988554,58.632548,0.453212,210.175146,10.000000,1,10.000000,1.044599,821.281092,0,0,98.249570,2
3,4,892.941177,0.159060,321.862350,638.076431,9.090909,0.820218,197.355744,4.363431,58.632548,5.505812,47.522528,6.343957,1,6.343957,1.145729,821.281092,1,0,140.862306,1
4,5,914.117647,0.083333,48.259929,626.514988,5.181818,1.372928,198.790477,85.938202,58.632548,0.174890,210.175146,14.717240,1,14.717240,1.558341,821.281092,1,0,101.268503,0


In [45]:
x_train_t=t_data.iloc[:,0:20]
y_train_t=t_data['VAR21']
x_train_p=pt_data.iloc[:,0:20]
y_train_p=pt_data['VAR21']

In [46]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import GridSearchCV  
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn import metrics

In [47]:
target = 'VAR21'
def modelfit(alg, dtrain, predictors,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgb_param['num_class']=3
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='mlogloss', early_stopping_rounds=early_stopping_rounds)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain['VAR21'],eval_metric='mlogloss')
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
        
    #Print model report:
    print ("\nModel Report")
    print ("Accuracy : %.4g" % metrics.accuracy_score(dtrain['VAR21'].values, dtrain_predictions))
#     print ("AUC Score (Train): %f" % metrics.roc_auc_score(dtrain['VAR21'], dtrain_predprob))
                    
#     feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
#     feat_imp.plot(kind='bar', title='Feature Importances')
#     plt.ylabel('Feature Importance Score')

In [48]:
predictors = [x for x in x_train_t.columns if x not in [target]]

In [ ]:
model1_t = XGBClassifier(
 learning_rate =0.01,
 n_estimators=1000,
 max_depth=6,
 min_child_weight=5,
 gamma=0.4,
 subsample=0.9,
 colsample_bytree=0.8,
 reg_alpha = 1.2,
 objective= 'multi:softmax',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
modelfit(model1_t, X_train_t, predictors)




In [ ]:
model1_p = XGBClassifier(
 learning_rate =0.01,
 n_estimators=1000,
 max_depth=6,
 min_child_weight=5,
 gamma=0.4,
 subsample=0.9,
 colsample_bytree=0.8,
 reg_alpha = 1.2,
 objective= 'multi:softmax',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
modelfit(model1_p, X_train_p, predictors)

In [ ]:
from xgboost import XGBClassifier
model2_t=XGBClassifier()
model2_t = model2_t.fit(x_train_t, y_train_t)

model2_p=XGBClassifier()
model2_p = model2_p.fit(x_train_p, y_train_p)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
model3_t=GradientBoostingClassifier()
model3_t=model3_t.fit(x_train_t,y_train_t)

model3_p=GradientBoostingClassifier()
model3_p=model3_p.fit(x_train_p,y_train_p)

In [ ]:
from sklearn import tree
model4_t = tree.DecisionTreeClassifier()
model4_t = model4_t.fit(x_train_t, y_train_t)

model4_p = tree.DecisionTreeClassifier()
model4_p = model4_p.fit(x_train_p, y_train_p)

In [ ]:
pred1_t=model1_t.predict(x_test)
pred2_t=model2_t.predict(x_test)
pred3_t=model3_t.predict(x_test)
pred4_t=model4_t.predict(x_test)

pred1_p=model1_p.predict(x_test)
pred2_p=model2_p.predict(x_test)
pred3_p=model3_p.predict(x_test)
pred4_p=model4_p.predict(x_test)

In [ ]:
f1=pd.DataFrame(pred1_t)
f2=pd.DataFrame(pred2_t)
f3=pd.DataFrame(pred3_t)
f4=pd.DataFrame(pred4_t)
f5=pd.DataFrame(pred1_p)
f6=pd.DataFrame(pred2_p)
f7=pd.DataFrame(pred3_p)
f8=pd.DataFrame(pred4_p)


f1=f1.astype('int64')
f2=f2.astype('int64')
f3=f3.astype('int64')
f4=f4.astype('int64')
f5=f5.astype('int64')
f6=f6.astype('int64')
f7=f7.astype('int64')
f8=f8.astype('int64')

col1=le.inverse_transform(f1)
col2=le.inverse_transform(f2)
col3=le.inverse_transform(f3)
col4=le.inverse_transform(f4)
col5=le.inverse_transform(f5)
col6=le.inverse_transform(f6)
col7=le.inverse_transform(f7)
col8=le.inverse_transform(f8)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:273: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
test = l_data['VAR1']

data_1 = { "num" : test,
        "COL1" : col1}
data_2 = { "num" : test,
        "COL2" : col2}
data_3 = { "num" : test,
        "COL3" : col3}
data_4 = { "num" : test,
        "COL4" : col4}
data_5 = { "num" : test,
        "COL5" : col5}
data_6 = { "num" : test,
        "COL6" : col6}
data_7 = { "num" : test,
        "COL7" : col7}
data_8 = { "num" : test,
        "COL8" : col8}


fu1 = pd.DataFrame(data_1)
fu2 = pd.DataFrame(data_2)
fu3 = pd.DataFrame(data_3)
fu4 = pd.DataFrame(data_4)
fu5 = pd.DataFrame(data_5)
fu6 = pd.DataFrame(data_6)
fu7 = pd.DataFrame(data_7)
fu8 = pd.DataFrame(data_8)

In [ ]:
fu=pd.concat([fu1.iloc[:,:1],fu1.iloc[:,1:],fu2.iloc[:,1:],fu3.iloc[:,1:],fu4.iloc[:,1:],fu5.iloc[:,1:],fu6.iloc[:,1:],fu7.iloc[:,1:],fu8.iloc[:,1:]],axis=1) 

In [ ]:
fu.head()

,num,COL1,COL2,COL3,COL4,COL5,COL6,COL7,COL8
0,1,Medium,Medium,Medium,Medium,Medium,Medium,High,Medium
1,2,Low,Low,Low,Low,Low,Low,Low,Low
2,3,Medium,Low,Low,High,Low,Medium,Medium,Low
3,4,Low,Low,Low,Low,Low,Low,Low,Medium
4,5,Medium,Medium,Low,Medium,Low,Low,Low,High


In [ ]:
fu.to_csv('/content/drive/My Drive/amex_2019/chips_IITKharagpur_sdf.csv',header=True, index=True)

In [ ]:
opts = ['COL{}'.format(i) for i in range(1, 9)]
fo=fu[opts].mode(axis=1)

In [ ]:
f0=fo.iloc[:,:1]
fo.head()

,0
0,Medium
1,Low
2,Low
3,Low
4,Low


In [ ]:
fo.to_csv('/content/drive/My Drive/amex_2019/chips_IITKharagpur_50.csv',header=False, index=False)

In [ ]:
fuck1.to_csv('/content/drive/My Drive/amex_2019/chips_IITKharagpur_50.csv',header=False, index=False)
fuck2.to_csv('/content/drive/My Drive/amex_2019/chips_IITKharagpur_51.csv',header=False, index=False)
fuck3.to_csv('/content/drive/My Drive/amex_2019/chips_IITKharagpur_52.csv',header=False, index=False)
fuck4.to_csv('/content/drive/My Drive/amex_2019/chips_IITKharagpur_53.csv',header=False, index=False)
fuck5.to_csv('/content/drive/My Drive/amex_2019/chips_IITKharagpur_54.csv',header=False, index=False)
fuck6.to_csv('/content/drive/My Drive/amex_2019/chips_IITKharagpur_55.csv',header=False, index=False)
fuck7.to_csv('/content/drive/My Drive/amex_2019/chips_IITKharagpur_56.csv',header=False, index=False)
fuck8.to_csv('/content/drive/My Drive/amex_2019/chips_IITKharagpur_57.csv',header=False, index=False)

In [ ]:
from statistics import mode

In [ ]:
import random
final_pred = np.array([])
for i in range(0,len(x_test)):
    final_pred = np.append(final_pred, random.choice(mode([pred1_p[i], pred2_p[i], pred3_p[i], pred4_p[i]])))

StatisticsError: ignored

In [ ]:
f=pd.DataFrame(final_pred)
f=f.astype('int64')
col1=le.inverse_transform(f)

In [ ]:
test = l_data['VAR1']
data  = { "col1" : test,
        "COL1" : col1}
fuck = pd.DataFrame(data)
fuck.head()

In [ ]:
fuck.to_csv('/content/drive/My Drive/amex_2019/chips_IITKharagpur_3.csv',header=False, index=False)